In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from ydata_profiling import ProfileReport
import numpy as np
import keras
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback, WandbMetricsLogger
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras import layers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
%matplotlib inline

2023-12-07 09:50:30.181955: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-07 09:50:30.183782: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-07 09:50:30.216882: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 09:50:30.216928: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 09:50:30.216956: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
df = pd.read_csv("/home/adi_senku/Downloads/parkinsons+telemonitoring/parkinsons_updrs.data")

In [ ]:
report = ProfileReport(df)

In [ ]:
report

In [ ]:
df.head()

In [ ]:
plt.hist(df['motor_UPDRS'].apply(np.sqrt), bins=15, histtype='step')

In [ ]:
plt.hist(df['Jitter(%)'].apply(np.log10), histtype='step')

In [3]:
columns = df.columns
print(columns)

Index(['subject#', 'age', 'sex', 'test_time', 'motor_UPDRS', 'total_UPDRS',
       'Jitter(%)', 'Jitter(Abs)', 'Jitter:RAP', 'Jitter:PPQ5', 'Jitter:DDP',
       'Shimmer', 'Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'Shimmer:APQ11', 'Shimmer:DDA', 'NHR', 'HNR', 'RPDE', 'DFA', 'PPE'],
      dtype='object')


In [4]:
columns_with_log = columns[6:-4]

In [5]:
for column in columns_with_log:
    df[f"{column}_log"] = df[column].apply(np.log10)

In [6]:
df_log = df.copy()

In [7]:
df_log.drop(columns_with_log, axis=1, inplace=True)

In [8]:
df_log.drop("subject#", axis=1, inplace=True)

In [9]:
y = df_log['total_UPDRS']
X = df_log.drop("total_UPDRS", axis=1)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
X_val, __, y_val, _ = train_test_split(X_train, y_train, test_size=0.8, random_state=42)

In [12]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

((4700, 20), (4700,), (1175, 20), (1175,), (940, 20), (940,))

In [13]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

In [14]:
def get_model():
    input = keras.Input((20,))
    hidden_11 = layers.Dense(128, activation='selu', kernel_initializer='lecun_normal')(input)
    hidden_21 = layers.Dense(128, activation='selu', kernel_initializer='lecun_normal')(input)
    concat_1 = layers.Concatenate()([hidden_11, hidden_21])

    hidden_12 = layers.Dense(64, activation='selu', kernel_initializer='lecun_normal')(concat_1)
    hidden_22 = layers.Dense(64, activation='selu', kernel_initializer='lecun_normal')(concat_1)
    concat_2 = layers.Concatenate()([hidden_12, hidden_22])

    hidden_13 = layers.Dense(32, activation='selu', kernel_initializer='lecun_normal')(concat_2)
    hidden_23 = layers.Dense(32, activation='selu', kernel_initializer='lecun_normal')(concat_2)
    concat_3 = layers.Concatenate()([hidden_13, hidden_23])

    hidden_14 = layers.Dense(16, activation='selu', kernel_initializer='lecun_normal')(concat_3)
    hidden_24 = layers.Dense(16, activation='selu', kernel_initializer='lecun_normal')(concat_3)
    concat_4 = layers.Concatenate()([hidden_14, hidden_24])

    hidden_15 = layers.Dense(8, activation='selu', kernel_initializer='lecun_normal')(concat_4)
    hidden_25 = layers.Dense(8, activation='selu', kernel_initializer='lecun_normal')(concat_4)
    concat_5 = layers.Concatenate()([hidden_15, hidden_25])

    hidden_16 = layers.Dense(4, activation='selu', kernel_initializer='lecun_normal')(concat_5)
    hidden_26 = layers.Dense(4, activation='selu', kernel_initializer='lecun_normal')(concat_5)
    concat_6 = layers.Concatenate()([hidden_16, hidden_26])

    hidden_17 = layers.Dense(2, activation='selu', kernel_initializer='lecun_normal')(concat_6)
    hidden_27 = layers.Dense(2, activation='selu', kernel_initializer='lecun_normal')(concat_6)
    concat_7 = layers.Concatenate()([hidden_17, hidden_27])

    output_1 = layers.Dense(1, activation='linear')(concat_7)
    output_2 = layers.Dense(1, activation='linear')(concat_7)

    add = layers.Concatenate()([output_1, output_2])
    add = layers.Average()([add, output_2, output_1])
    add = layers.Dense(1)(add)
    model = keras.Model(inputs=[input], outputs=[add])
    return model

In [15]:
model = get_model()

In [16]:
model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=0.003), metrics=['mse'])

In [ ]:
wandb.login()

In [ ]:
wandb.init(project='Parkinson')

In [22]:
model.fit(X_train_scaled, y_train, epochs=500,
          validation_data=(X_val_scaled, y_val), callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

Epoch 1/500
147/147 [==============================] - 0s 3ms/step - loss: 0.7136 - mse: 0.7136 - val_loss: 1.1212 - val_mse: 1.1212
Epoch 2/500
147/147 [==============================] - 0s 3ms/step - loss: 0.9046 - mse: 0.9046 - val_loss: 0.7307 - val_mse: 0.7307
Epoch 3/500
147/147 [==============================] - 0s 2ms/step - loss: 0.9152 - mse: 0.9152 - val_loss: 0.7596 - val_mse: 0.7596
Epoch 4/500
147/147 [==============================] - 0s 2ms/step - loss: 1.2923 - mse: 1.2923 - val_loss: 1.0142 - val_mse: 1.0142
Epoch 5/500
147/147 [==============================] - 0s 2ms/step - loss: 1.1153 - mse: 1.1153 - val_loss: 1.6206 - val_mse: 1.6206
Epoch 6/500
147/147 [==============================] - 0s 2ms/step - loss: 0.8608 - mse: 0.8608 - val_loss: 0.6502 - val_mse: 0.6502
Epoch 7/500
147/147 [==============================] - 0s 3ms/step - loss: 0.7145 - mse: 0.7145 - val_loss: 0.4618 - val_mse: 0.4618
Epoch 8/500
147/147 [==============================] - 0s 3ms/step - 

In [17]:
best_model = keras.models.load_model("model-best.h5")

In [18]:
pred = best_model.predict(X_test_scaled)

37/37 [==============================] - 0s 2ms/step


In [ ]:
pred.shape, y_test.shape

In [19]:
df_pred = pd.DataFrame({"Actual":y_test, "Predicted":tf.squeeze(pred)})

In [ ]:
i = 1
for pred_value in pred:
    df_pred[f"Model_{i}_pred"] = pred_value
    i += 1

In [22]:
df_pred.head()

,Actual,Predicted,Error
5366,43.563,43.522488,0.001641
4326,12.060,12.027708,0.001043
199,14.109,14.132563,0.000555
3260,16.517,15.903656,0.376191
4955,32.000,31.751707,0.061649


In [20]:
df_pred["Error"] = np.square((df_pred["Actual"] - df_pred["Predicted"]))

In [21]:
np.average(df_pred["Error"])

0.17866294500651583

In [16]:
def train_model():
    wandb.init()
    config = wandb.config
    
    input = keras.Input((20,))
    hidden_11 = layers.Dense(units=config.hidden_layers, activation=config.activation, kernel_initializer='lecun_normal')(input)
    hidden_21 = layers.Dense(units=config.hidden_layers, activation=config.activation, kernel_initializer='lecun_normal')(input)
    concat_1 = layers.Concatenate()([hidden_11, hidden_21])

    hidden_12 = layers.Dense(64, activation=config.activation, kernel_initializer='lecun_normal')(concat_1)
    hidden_22 = layers.Dense(64, activation=config.activation, kernel_initializer='lecun_normal')(concat_1)
    concat_2 = layers.Concatenate()([hidden_12, hidden_22])

    hidden_13 = layers.Dense(32, activation=config.activation, kernel_initializer='lecun_normal')(concat_2)
    hidden_23 = layers.Dense(32, activation=config.activation, kernel_initializer='lecun_normal')(concat_2)
    concat_3 = layers.Concatenate()([hidden_13, hidden_23])

    hidden_14 = layers.Dense(16, activation=config.activation, kernel_initializer='lecun_normal')(concat_3)
    hidden_24 = layers.Dense(16, activation=config.activation, kernel_initializer='lecun_normal')(concat_3)
    concat_4 = layers.Concatenate()([hidden_14, hidden_24])

    hidden_15 = layers.Dense(8, activation=config.activation, kernel_initializer='lecun_normal')(concat_4)
    hidden_25 = layers.Dense(8, activation=config.activation, kernel_initializer='lecun_normal')(concat_4)
    concat_5 = layers.Concatenate()([hidden_15, hidden_25])

    hidden_16 = layers.Dense(4, activation=config.activation, kernel_initializer='lecun_normal')(concat_5)
    hidden_26 = layers.Dense(4, activation=config.activation, kernel_initializer='lecun_normal')(concat_5)
    concat_6 = layers.Concatenate()([hidden_16, hidden_26])

    hidden_17 = layers.Dense(2, activation=config.activation, kernel_initializer='lecun_normal')(concat_6)
    hidden_27 = layers.Dense(2, activation=config.activation, kernel_initializer='lecun_normal')(concat_6)
    concat_7 = layers.Concatenate()([hidden_17, hidden_27])

    output_1 = layers.Dense(1, activation='linear')(concat_7)
    output_2 = layers.Dense(1, activation='linear')(concat_7)

    add = layers.Concatenate()([output_1, output_2])
    add = layers.Average()([output_2, output_1, add])
    add = layers.Dense(1)(add)
    model = keras.Model(inputs=input, outputs=[add])

    model.compile(loss='mse', optimizer=Adam(learning_rate=config.learning_rate), metrics=['mse', 'mape', 'mae', keras.metrics.RootMeanSquaredError()])

    model.fit(X_train_scaled, y_train,
              validation_data=(X_val_scaled, y_val),
              epochs=config.epochs,
              callbacks=[WandbMetricsLogger(log_freq=2), EarlyStopping(patience=10, restore_best_weights=True),
                         WandbCallback(validation_data=(X_val_scaled, y_val), save_model=False, save_weights_only=True, mode='min', monitor='val_loss'), keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.3, min_delta=0.0002, patience=5)])

In [14]:
sweep_config = {
    'method':'random',
    'name': "Random with just weights",
    'metric':{
        'name':'val_mse',
        'goal':'minimize'
    },
    'parameters':{
        'epochs':{
            'values':[200, 350, 500, 650]
        },
        'hidden_layers':{
            'values':[128, 256, 512]
        },
        'activation':{
            'values':['relu', 'selu']
        },
        'learning_rate':{
            "max":0.003, "min":0.3e-5
        }
    }
}

In [15]:
sweep_id = wandb.sweep(sweep_config, project='Parkinson')

Create sweep with ID: hrkzw7bd
Sweep URL: https://wandb.ai/the-idli-boys/Parkinson/sweeps/hrkzw7bd


In [ ]:
wandb.agent(sweep_id, function=train_model)

wandb: Agent Starting Run: 2m5i90w6 with config:
wandb: 	activation: relu
wandb: 	epochs: 500
wandb: 	hidden_layers: 512
wandb: 	learning_rate: 0.0002219600078994952
wandb: Currently logged in as: vaditya06 (the-idli-boys). Use `wandb login --relogin` to force relogin


Epoch 1/500
147/147 [==============================] - 2s 5ms/step - loss: 537.6147 - mse: 537.6147 - mape: 68.3734 - mae: 19.4859 - root_mean_squared_error: 23.1865 - val_loss: 181.2666 - val_mse: 181.2666 - val_mape: 40.6652 - val_mae: 10.4711 - val_root_mean_squared_error: 13.4635 - lr: 2.2196e-04
Epoch 2/500
147/147 [==============================] - 1s 4ms/step - loss: 93.4899 - mse: 93.4899 - mape: 29.7581 - mae: 7.4977 - root_mean_squared_error: 9.6690 - val_loss: 45.4559 - val_mse: 45.4559 - val_mape: 21.4587 - val_mae: 5.3544 - val_root_mean_squared_error: 6.7421 - lr: 2.2196e-04
Epoch 3/500
147/147 [==============================] - 1s 4ms/step - loss: 34.2253 - mse: 34.2253 - mape: 18.5197 - mae: 4.6071 - root_mean_squared_error: 5.8502 - val_loss: 27.6648 - val_mse: 27.6648 - val_mape: 16.9067 - val_mae: 4.1512 - val_root_mean_squared_error: 5.2597 - lr: 2.2196e-04
Epoch 4/500
147/147 [==============================] - 1s 4ms/step - loss: 25.1457 - mse: 25.1457 - mape: 16.1

batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,█████████████████▃▃▃▃▃▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/mae,█▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/mape,█▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/mse,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/root_mean_squared_error,█▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,█████████████████▃▃▃▃▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: Agent Starting Run: 69o27f39 with config:
wandb: 	activation: selu
wandb: 	epochs: 350
wandb: 	hidden_layers: 256
wandb: 	learning_rate: 0.0022821058288146523


Epoch 1/350
147/147 [==============================] - 2s 3ms/step - loss: 162.9926 - mse: 162.9926 - mape: 30.5599 - mae: 8.1317 - root_mean_squared_error: 12.7669 - val_loss: 13.3023 - val_mse: 13.3023 - val_mape: 11.0990 - val_mae: 2.8976 - val_root_mean_squared_error: 3.6472 - lr: 0.0023
Epoch 2/350
147/147 [==============================] - 0s 3ms/step - loss: 9.6739 - mse: 9.6739 - mape: 9.3966 - mae: 2.3912 - root_mean_squared_error: 3.1103 - val_loss: 7.8145 - val_mse: 7.8145 - val_mape: 8.1066 - val_mae: 2.1539 - val_root_mean_squared_error: 2.7955 - lr: 0.0023
Epoch 3/350
147/147 [==============================] - 0s 3ms/step - loss: 7.3714 - mse: 7.3714 - mape: 8.2465 - mae: 2.0880 - root_mean_squared_error: 2.7150 - val_loss: 5.7048 - val_mse: 5.7048 - val_mape: 7.1958 - val_mae: 1.8363 - val_root_mean_squared_error: 2.3885 - lr: 0.0023
Epoch 4/350
147/147 [==============================] - 0s 3ms/step - loss: 5.5298 - mse: 5.5298 - mape: 6.8383 - mae: 1.7757 - root_mean_sq

...

wandb: Agent Starting Run: hd8kf6w9 with config:
wandb: 	activation: selu
wandb: 	epochs: 650
wandb: 	hidden_layers: 128
wandb: 	learning_rate: 0.0017813841540734968


Epoch 1/650
147/147 [==============================] - 2s 3ms/step - loss: 317.6580 - mse: 317.6580 - mape: 46.4670 - mae: 13.0039 - root_mean_squared_error: 17.8230 - val_loss: 19.8365 - val_mse: 19.8365 - val_mape: 15.1131 - val_mae: 3.5875 - val_root_mean_squared_error: 4.4538 - lr: 0.0018
Epoch 2/650
147/147 [==============================] - 0s 2ms/step - loss: 13.4514 - mse: 13.4514 - mape: 11.5820 - mae: 2.8660 - root_mean_squared_error: 3.6676 - val_loss: 9.7332 - val_mse: 9.7332 - val_mape: 9.9314 - val_mae: 2.4291 - val_root_mean_squared_error: 3.1198 - lr: 0.0018
Epoch 3/650
147/147 [==============================] - 0s 2ms/step - loss: 9.5499 - mse: 9.5499 - mape: 9.1986 - mae: 2.3875 - root_mean_squared_error: 3.0903 - val_loss: 7.6949 - val_mse: 7.6949 - val_mape: 7.8696 - val_mae: 2.1106 - val_root_mean_squared_error: 2.7740 - lr: 0.0018
Epoch 4/650
147/147 [==============================] - 0s 2ms/step - loss: 8.3234 - mse: 8.3234 - mape: 8.4996 - mae: 2.2230 - root_mea

batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,███████▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/mae,█▆▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/mape,█▆▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/mse,█▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/root_mean_squared_error,█▆▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,███████▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


wandb: Agent Starting Run: pkbw2nqm with config:
wandb: 	activation: relu
wandb: 	epochs: 200
wandb: 	hidden_layers: 256
wandb: 	learning_rate: 0.001501719622690965


Epoch 1/200
147/147 [==============================] - 2s 3ms/step - loss: 371.9044 - mse: 371.9044 - mape: 51.9025 - mae: 14.4162 - root_mean_squared_error: 19.2848 - val_loss: 27.7361 - val_mse: 27.7361 - val_mape: 17.0376 - val_mae: 4.2527 - val_root_mean_squared_error: 5.2665 - lr: 0.0015
Epoch 2/200
147/147 [==============================] - 0s 2ms/step - loss: 20.1613 - mse: 20.1613 - mape: 14.7640 - mae: 3.5719 - root_mean_squared_error: 4.4901 - val_loss: 14.7490 - val_mse: 14.7490 - val_mape: 13.0711 - val_mae: 3.0573 - val_root_mean_squared_error: 3.8404 - lr: 0.0015
Epoch 3/200
147/147 [==============================] - 0s 3ms/step - loss: 13.2472 - mse: 13.2472 - mape: 11.9416 - mae: 2.8835 - root_mean_squared_error: 3.6397 - val_loss: 10.3961 - val_mse: 10.3961 - val_mape: 10.5282 - val_mae: 2.5435 - val_root_mean_squared_error: 3.2243 - lr: 0.0015
Epoch 4/200
147/147 [==============================] - 0s 3ms/step - loss: 9.6476 - mse: 9.6476 - mape: 10.1034 - mae: 2.4456 